<a href="https://colab.research.google.com/github/acastellanos-ie/natural_language_processing/blob/master/tagging_parsing_practice/arabic_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Configuration

**Execute this steps to configure the Google Colab environment in order to execute this notebook. It is not required if you are executing it locally and you have properly configured your local environment according to what explained in the Github Repository.**

The first step is to clone the repository to have access to all the data and files

In [ ]:
! git clone https://github.com/acastellanos-ie/natural_language_processing.git

Cloning into 'natural_language_processing'...
fatal: could not read Username for 'https://github.com': No such device or address


Install the requirements

In [ ]:
! pip install -Uqqr natural_language_processing/arabic_requirements.txt

Now you have everything you need to execute the code in Colab

In order to use all the components provided in CAMeL Tools, we need to install all the datasets required by these components.
To do this in Colab, we need to first mount a Google Drive and create a directory where the data will be installed.

Run the code below and follow the instructions in the output.

In [ ]:
from google.colab import drive
import os

drive.mount('/gdrive')

%mkdir /gdrive/MyDrive/camel_tools

Next, we need to tell CAMeL Tools to install the data in the newly created directory. This will take a couple of minutes to complete.

**NOTE:** You will need at least 2.3GB of available space on your Google Drive to install all the CAMeL Tools data.

In [ ]:
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

!export | camel_data full

# Introduction

This notebook provides a quick overview of the different pre-processing steps needed to deal with textual contents in Arabic. It does not intend to be a full review but to provide you with illustrative examples and pointers to additional materials.

To that end, I have used the guidelines and functionalities provided by [CAMeL Tools](https://github.com/CAMeL-Lab/camel_tools), one of the libraries that I recommended you to check.

For more detailed information please refer to the [CAMeL Tools documentation](https://camel-tools.readthedocs.io/en/latest/index.html).

# Dediacritization

Dediacritization is the process of removing Arabic diacritical marks. 

As discussed in class, Diacritics increase data sparsity and so most Arabic NLP techniques ignore them. The example below shows how diacritics can be removed from Arabic text using the [`dediac_ar`](https://camel-tools.readthedocs.io/en/latest/api/utils/dediac.html#camel_tools.utils.dediac.dediac_ar) function:

In [ ]:
from camel_tools.utils.dediac import dediac_ar

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_dediac = dediac_ar(sentence)
print(sent_dediac)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
هل ذهبت إلى المكتبة؟


# Tokenization



## Word Tokenization

Tokenization is the step to split the entire sentence into the individual units of meaning (i.e., words). In class, we reviewed the standard process and the challenges it posed to Arabic.

In this sense, there are different tokenization strategies that we can follow.

Python does provide the `split()` method to tokenize words by whitespace, but it doesn't separate punctuation from words. For example:

In [ ]:
sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = sentence.split()
print(sent_split)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ؟']


A similar function to split by whitespace and separate punctuation is also provided by CAMeL Tools [`simple_word_tokenize`](https://camel-tools.readthedocs.io/en/latest/api/tokenizers/word.html#camel_tools.tokenizers.word.simple_word_tokenize).

The example below is similar to the one above, but this time we use `simple_word_tokenize()` instead of `split()`.

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize

sentence = "هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟"
print(sentence)

sent_split = simple_word_tokenize(sentence)
print(sent_split)

هَلْ ذَهَبْتَ إِلَى المَكْتَبَةِ؟
['هَلْ', 'ذَهَبْتَ', 'إِلَى', 'المَكْتَبَةِ', '؟']


## Morphological Tokenization

The tokenization strategies mentioned above are simply splitting sentences by whitespace and separating punctuation. However, this is not the best choice for Arabic. In contrast, morphological tokenization splits Arabic words into component prefixes, stems, and suffixes.

CAMeL Tools provides the [`MorphologicalTokenizer`](https://camel-tools.readthedocs.io/en/latest/api/tokenizers/morphological.html#camel_tools.tokenizers.morphological.MorphologicalTokenizer) class to tokenize words in different schemes. It behaves very much like the `DefaultTagger` in that it uses a disambiguator first to disambiguate words and then extracts a particular tokenization feature, but it has the following differences:

- While the `DefaultTagger` produces exactly one output for each input word, the `MorphologicalTokenizer` might produce multiple output tokens.
-  The `MorphologicalTokenizer` can be configured to produce diacritized and undiacritized output.

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer

# The tokenizer expects pre-tokenized text
sentence = simple_word_tokenize('فتنفست الصعداء')
print(sentence)

# Load a pretrained disambiguator to use with a tokenizer
mle = MLEDisambiguator.pretrained('calima-msa-r13')

# Without providing additional arguments, the tokenizer will output undiacritized
# morphological tokens for each input word delimited by an underscore.
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok')
tokens = tokenizer.tokenize(sentence)
print(tokens)

# By specifying `split=True`, the morphological tokens are output as seperate
# strings.
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)
tokens = tokenizer.tokenize(sentence)
print(tokens)

# We can output diacritized tokens by setting `diac=True`
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True, diac=True)
tokens = tokenizer.tokenize(sentence)
print(tokens)

['فتنفست', 'الصعداء']
['ف+_تنفست', 'ال+_صعداء']
['ف+', 'تنفست', 'ال+', 'صعداء']
['فَ+', 'تَنَفَّسْتُ', 'ال+', 'صُعَداءَ']


# Tagging

This step focuses on identifying the role of a given word in the context of a particular sentence. 

CAMeL Tools provides a [`DefaultTagger`](https://camel-tools.readthedocs.io/en/latest/api/tagger/default.html#camel_tools.tagger.default.DefaultTagger) for the POS Tagging of Arabic contents.

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mle = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mle, 'pos')

# The tagger expects pre-tokenized text
sentence = simple_word_tokenize('نجح بايدن في الانتخابات')

pos_tags = tagger.tag(sentence)

print(pos_tags)

['verb', 'noun_prop', 'prep', 'noun']


# Named Entitiy Recognition

Finally, CAMeL Tools comes with an easy-to-use, pretrained [named-entitity recognition](https://en.wikipedia.org/wiki/Named-entity_recognition)  (NER) system that can be accessed using the [`NERecognizer`](https://camel-tools.readthedocs.io/en/latest/api/ner.html#camel_tools.ner.NERecognizer) class. 

For each token in an input sentence, `NERecognizer` outputs a label that indicates the type of named-entity. The system outputs one of the following labels for each token: `'B-LOC'`, `'B-ORG'`, `'B-PERS'`, `'B-MISC'`, `'I-LOC'`, `'I-ORG'`, `'I-PERS'`, `'I-MISC'`, `'O'`.
Named-entites can either be a `LOC` (location), `ORG` (organization), `PERS` (person), or `MISC` (miscallaneous).

Labels beginning with `B` indicate that their corresponding tokens are the beginning of a multi-word named-entity or are a single-token named-entity'. Those begining with `I` indicate that their corresponding tokens are continuations of a multi-word named-entity. Words that aren't named-entities are given the `'O'` label.

The example below illustrates how `NERecognizer` can be used to label named-entities in a given sentence.


In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.ner import NERecognizer

ner = NERecognizer.pretrained()

# NERecognizer expects pre-tokenized text
sentence = simple_word_tokenize('إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع.')

labels = ner.predict_sentence(sentence)

# Print each token paired with it's NER label
print(list(zip(sentence, labels)))

[('إمارة', 'O'), ('أبوظبي', 'B-LOC'), ('هي', 'O'), ('إحدى', 'O'), ('إمارات', 'O'), ('دولة', 'O'), ('الإمارات', 'B-LOC'), ('العربية', 'I-LOC'), ('المتحدة', 'I-LOC'), ('السبع', 'O'), ('.', 'O')]
